Lambda School Data Science

*Unit 2, Sprint 3, Module 3*

---


# Permutation & Boosting

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your work.

- [ ] If you haven't completed assignment #1, please do so first.
- [ ] Continue to clean and explore your data. Make exploratory visualizations.
- [ ] Fit a model. Does it beat your baseline? 
- [ ] Try xgboost.
- [ ] Get your model's permutation importances.

You should try to complete an initial model today, because the rest of the week, we're making model interpretation visualizations.

But, if you aren't ready to try xgboost and permutation importances with your dataset today, that's okay. You can practice with another dataset instead. You may choose any dataset you've worked with previously.

The data subdirectory includes the Titanic dataset for classification and the NYC apartments dataset for regression. You may want to choose one of these datasets, because example solutions will be available for each.


## Reading

Top recommendations in _**bold italic:**_

#### Permutation Importances
- _**[Kaggle / Dan Becker: Machine Learning Explainability](https://www.kaggle.com/dansbecker/permutation-importance)**_
- [Christoph Molnar: Interpretable Machine Learning](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

#### (Default) Feature Importances
  - [Ando Saabas: Selecting good features, Part 3, Random Forests](https://blog.datadive.net/selecting-good-features-part-iii-random-forests/)
  - [Terence Parr, et al: Beware Default Random Forest Importances](https://explained.ai/rf-importance/index.html)

#### Gradient Boosting
  - [A Gentle Introduction to the Gradient Boosting Algorithm for Machine Learning](https://machinelearningmastery.com/gentle-introduction-gradient-boosting-algorithm-machine-learning/)
  - [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf), Chapter 8
  - _**[Gradient Boosting Explained](https://www.gormanalysis.com/blog/gradient-boosting-explained/)**_ — Ben Gorman
  - [Gradient Boosting Explained](http://arogozhnikov.github.io/2016/06/24/gradient_boosting_explained.html) — Alex Rogozhnikov
  - [How to explain gradient boosting](https://explained.ai/gradient-boosting/) — Terence Parr & Jeremy Howard

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from category_encoders import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

df_raw = pd.read_csv('../data/msrp.csv')

# EDA

In [ ]:
df_raw.head()

In [ ]:
## Target Variable Distribution (regression)
df_raw['MSRP'].hist(bins=100)

In [ ]:
df_raw.plot.scatter('MSRP', 'Engine HP')

In [ ]:
df_raw.select_dtypes('object').nunique().sort_values()

# Wrangle Data

- Wrangle According to Visualizations Above

In [ ]:
## Wrangle function from Assign21
def wrangle(df):
    df = df.copy()
    
    ## Fix column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
    ## Remove Outliers
    df = df[df['msrp'] <= 75000]
    
    ## Reduce high cardinality in 'make'
    df['make'] = [x if x in list(df['make'].value_counts().sort_values().tail(5).index) else 'Other' for x in df['make']]
    
    ## Reduce high cardinality in 'market_category' (maybe remove NaN's)
    df['luxury'] = [x if type(x)==float else 1 if 'Luxury' in x else 0 for x in df['market_category']]
    
    ## 'Model' is going to allow for leakage in a way
    df.drop(columns=['model', 'market_category'], inplace=True)
    
    return df

In [ ]:
df = wrangle(df_raw)

In [ ]:
df.head()

# Split Data

- Target Variable
- Feature Matrix
- Train-Test Split

In [ ]:
target = 'msrp'
y = df[target]
X = df.drop(columns=target)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build Model

In [ ]:
linreg = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    LinearRegression()
)

linreg.fit(X_train, y_train);

In [ ]:
xgb = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    XGBClassifier(random_state=42)
)

xgb.fit(X_train, y_train);

# Check Metrics

In [ ]:
y_pred = [y_train.mean()]*len(y_train)
print('Baseline MAE:', mean_absolute_error(y_train, y_pred))
print('LinReg Training MAE:', mean_absolute_error(y_train, linreg.predict(X_train)))
print('LinReg Test MAE:', mean_absolute_error(y_test, linreg.predict(X_test)))

In [ ]:
print('XGB Training MAE:', mean_absolute_error(y_train, xgb.predict(X_train)))
print('XGB Test MAE:', mean_absolute_error(y_test, xgb.predict(X_test)))